# Goal of the notebook

So far we have collected only data related to options themselves. However, in order to apply Black-Scholes model (and probably many other reasonable models), we need to have price of an underlying assets as inputs. Hence, the goal of this notebook is to merge our option data with stock data. We will also extract trading volumes of underlying assets as additional features.

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset_full = pd.read_csv('Maturity_data_02_05.csv')
dataset_full['Date_get_data'] = pd.to_datetime(dataset_full['Date_get_data'])
dataset_full['Date_expiration'] = pd.to_datetime(dataset_full['Date_expiration'])
dataset_full

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (3,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Company,Date_get_data,Date_expiration,Time_to_maturity,r
0,AAPL240223C00095000,95.0,86.97,87.1,87.6,0.00,-,1,7,177.34%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
1,AAPL240223C00100000,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,173.44%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
2,AAPL240223C00110000,110.0,83.50,72.1,72.65,0.00,-,-,1,148.05%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
3,AAPL240223C00115000,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,135.94%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
4,AAPL240223C00120000,120.0,64.40,62.1,62.65,0.00,-,6,3,124.61%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223321,INTC261218C00050000,50.0,5.70,5.55,5.8,-0.80,-12.31%,42,1420,42.66%,INTC,2024-04-12,2026-12-18,2.684932,0.04
1223322,INTC261218C00055000,55.0,4.80,4.7,4.8,-0.60,-11.11%,73,31727,42.29%,INTC,2024-04-12,2026-12-18,2.684932,0.04
1223323,INTC261218C00060000,60.0,3.95,3.8,4.05,-0.57,-12.61%,68,2618,42.29%,INTC,2024-04-12,2026-12-18,2.684932,0.04
1223324,INTC261218C00065000,65.0,3.30,2.52,3.35,-0.47,-12.47%,22,304,41.89%,INTC,2024-04-12,2026-12-18,2.684932,0.04


In [3]:
dataset_full.dtypes

Contract Name                 object
Strike                       float64
Last Price                   float64
Bid                           object
Ask                           object
Change                       float64
% Change                      object
Volume                        object
Open Interest                 object
Implied Volatility            object
Company                       object
Date_get_data         datetime64[ns]
Date_expiration       datetime64[ns]
Time_to_maturity             float64
r                            float64
dtype: object

### Get prices and trading volumes

In [4]:
import yfinance as yf
import datetime as dt
import pandas_datareader as pdr
pd.set_option('display.max_columns', None)

In [5]:
yf.pdr_override()

In [6]:
from pandas_datareader import data as pdr 
import matplotlib.pyplot as plt

In [7]:
start = "2020-01-01"
end = "2024-04-13"

In [8]:
symbols_stocks = ['AAPL', 
            'GOOGL',
            'MSFT',
            'SPY',
            'PFE',
            'LMT',
            'DIS',
            'VZ',
            'PYPL',
            'TSN',
            'DG',
            'QCOM',
            'META',
            'TSLA',
            'NFLX',
            'NVDA',
            'AMZN',
            'AVGO',
            'LLY',
            'JPM',
            'UNH',
            'V',
            'MA',
            'JNJ',
            'PG',
            'MRK',
            'MCD',
            'WMT',
            'BAC',
            'ACN',
            'KO',
            'PEP',
            'CSCO',
            'ORCL',
            'INTC'] 

data_stocks_close = pdr.get_data_yahoo(symbols_stocks, start, end)['Close']
data_stocks_volume = pdr.get_data_yahoo(symbols_stocks, start, end)['Volume']

[*********************100%%**********************]  35 of 35 completed
[*********************100%%**********************]  35 of 35 completed


In [9]:
# prices of underlying assets
data_stocks_close = data_stocks_close.reset_index()
data_stocks_close['Date'] = pd.to_datetime(data_stocks_close['Date'])
data_stocks_close

,Date,AAPL,ACN,AMZN,AVGO,BAC,CSCO,DG,DIS,GOOGL,INTC,JNJ,JPM,KO,LLY,LMT,MA,MCD,META,MRK,MSFT,NFLX,NVDA,ORCL,PEP,PFE,PG,PYPL,QCOM,SPY,TSLA,TSN,UNH,V,VZ,WMT
0,2020-01-02,75.087502,210.149994,94.900497,322.390015,35.639999,48.419998,156.539993,148.199997,68.433998,60.840000,145.970001,141.089996,54.990002,132.210007,399.369995,303.390015,200.789993,209.779999,87.824425,160.619995,329.809998,59.977501,53.950001,135.820007,37.134724,123.410004,110.750000,88.690002,324.869995,28.684000,89.970001,292.500000,191.119995,61.049999,39.646667
1,2020-01-03,74.357498,209.800003,93.748497,314.190002,34.900002,47.630001,155.389999,146.500000,68.075996,60.099998,144.279999,138.339996,54.689999,131.770004,413.739990,300.429993,200.080002,208.669998,87.070610,158.619995,325.899994,59.017502,53.759998,135.630005,36.935486,122.580002,108.760002,87.019997,322.410004,29.534000,90.580002,289.540009,189.600006,60.400002,39.296665
2,2020-01-06,74.949997,208.429993,95.143997,313.720001,34.849998,47.799999,154.539993,145.649994,69.890503,59.930000,144.100006,138.229996,54.669998,132.259995,413.109985,301.230011,202.330002,212.600006,87.442749,159.029999,335.829987,59.264999,54.040001,136.149994,36.888046,122.750000,110.169998,86.510002,323.640015,30.102667,90.150002,291.549988,189.190002,60.270000,39.216667
3,2020-01-07,74.597504,203.929993,95.343002,312.640015,34.619999,47.490002,152.210007,145.699997,69.755501,58.930000,144.979996,135.880005,54.250000,132.509995,414.500000,300.209991,202.630005,213.059998,85.114502,157.580002,330.750000,59.982498,54.160000,134.009995,36.764706,121.989998,109.669998,88.970001,322.730011,31.270666,88.839996,289.790009,188.690002,59.599998,38.853333
4,2020-01-08,75.797501,204.330002,94.598503,308.739990,34.970001,47.520000,151.550003,145.399994,70.251999,58.970001,144.960007,136.940002,54.349998,133.710007,411.029999,305.100006,205.910004,215.220001,84.541985,160.089996,339.260010,60.095001,54.130001,134.699997,37.058823,122.510002,111.820000,88.709999,324.450012,32.809334,89.389999,295.899994,191.919998,59.709999,38.720001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2024-04-08,168.449997,331.799988,185.190002,1336.099976,37.500000,48.240002,156.740005,117.349998,154.850006,37.980000,151.589996,198.479996,59.270000,777.289978,452.380005,478.850006,267.559998,519.250000,126.559998,424.589996,628.409973,871.330017,124.349998,169.580002,26.580000,156.039993,66.339996,173.619995,518.719971,172.979996,59.090000,456.000000,277.760010,41.730000,59.779999
1073,2024-04-09,169.669998,334.450012,185.669998,1334.079956,37.730000,50.009998,153.389999,117.970001,156.600006,38.330002,152.289993,197.149994,59.720001,757.239990,447.570007,472.160004,269.440002,516.900024,126.709999,426.279999,618.200012,853.539978,123.230003,170.610001,26.780001,156.660004,67.000000,175.570007,519.320007,176.880005,59.410000,459.720001,276.720001,40.840000,59.790001
1074,2024-04-10,167.779999,324.399994,185.949997,1322.369995,36.650002,49.209999,154.630005,117.190002,156.139999,37.200001,150.199997,195.470001,58.919998,761.979980,451.709991,469.170013,268.670013,519.830017,126.750000,423.260010,618.580017,870.390015,121.750000,168.949997,26.320000,157.240005,65.879997,170.860001,514.119995,171.759995,58.169998,450.049988,274.480011,40.520000,60.619999
1075,2024-04-11,175.039993,325.190002,189.050003,1382.459961,36.349998,49.529999,154.369995,117.150002,159.410004,37.630001,148.789993,195.429993,59.049999,759.590027,452.320007,468.309998,268.619995,523.159973,126.150002,427.929993,628.780029,906.159973,123.239998,168.360001,26.340000,155.839996,65.800003,175.130005,518.000000,174.600006,59.340000,441.720001,275.679993,40.160000,60.090000


In [10]:
# trading volumes
data_stocks_volume = data_stocks_volume.reset_index()
data_stocks_volume['Date'] = pd.to_datetime(data_stocks_volume['Date'])
data_stocks_volume

,Date,AAPL,ACN,AMZN,AVGO,BAC,CSCO,DG,DIS,GOOGL,INTC,JNJ,JPM,KO,LLY,LMT,MA,MCD,META,MRK,MSFT,NFLX,NVDA,ORCL,PEP,PFE,PG,PYPL,QCOM,SPY,TSLA,TSN,UNH,V,VZ,WMT
0,2020-01-02,135480400,2431100,80580000,2032900,37614200,16708100,1918500,9502100,27278000,18056000,5777000,10803700,11867700,2204200,1258400,3272400,3554200,12077100,8251428,22622100,4485800,23753600,13899600,3784100,16514072,8130800,7418800,8413900,59151200,142981500,2672200,2543400,8733000,11447900,20294700
1,2020-01-03,146322800,1802100,75288000,2287000,50357900,15577400,1715300,7320200,23408000,15293900,5752400,10386800,11354500,1963500,2990100,2501300,2767600,11188400,5903698,21116200,3806900,20538400,11026700,4000100,14922848,7970500,7098300,8340300,77709700,266677500,2506100,2711400,4899700,13263200,16197600
2,2020-01-06,118387200,2841400,81236000,2078900,42185000,22183600,2295600,8262500,46768000,17755200,7731300,10259000,14698300,2102900,2477800,3307200,4660400,17058900,7522963,20813700,5663100,26263600,10982400,4085100,15771951,6674400,6764500,8381400,55653900,151995000,2262500,3079100,10109500,15094500,19336500
3,2020-01-07,108872000,3097400,80898000,1856500,34149000,16501900,2832000,6906500,34330000,21876100,7382900,10531300,9973900,2448300,1059900,3182000,4047400,14912400,11132275,21634100,4703200,31485600,12015400,5718100,20108107,7583400,6898700,8377400,40496400,268231500,2088700,2492900,4392300,14293100,20540700
4,2020-01-08,132079200,2271300,70160000,3197900,45311600,25175900,3208400,6984200,35314000,23133500,6605800,9695300,10676000,5188600,1708100,3224400,5284200,13475000,15200506,27746500,7104500,27710800,11856700,3681400,16403507,5385100,8293600,7619900,68296000,467164500,2166400,3379200,5712000,15926500,17627400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2024-04-08,37425500,2431900,39221300,1713900,29470300,15375700,2637700,6266000,20702000,56967800,5935400,8001000,10284500,1664200,751100,1961800,3753200,13260600,6282500,14272400,2145700,28322000,6119200,3755700,24431400,5476800,8421400,4812000,48401800,104423300,1997000,4532300,5542400,14457800,11070100
1073,2024-04-09,42451200,2416600,36546900,1800800,30977700,26697100,2645800,7455400,31113000,78783000,6135600,7358600,10792100,2464800,842800,2218000,3934900,10881400,4696600,12512300,2146600,50354700,5564900,3774300,45193600,4613800,8459900,5030500,68124400,103232700,1715500,3521700,7314300,17227000,10015900
1074,2024-04-10,49709300,2156700,35879200,1746700,37610700,20026400,1884700,6746300,22838600,51108400,6809500,7681400,11861900,1700300,1247700,2710500,3817500,11418500,5787700,16216600,2806200,43192900,4509900,4581500,44541600,7784900,8566300,6177400,82652800,84532400,1403100,4493500,5276000,17900400,15097600
1075,2024-04-11,91070300,2204200,40020700,3171300,38604000,15703900,1791900,6559400,27166400,41909100,8374400,10137700,11109000,1594800,942800,2299900,2519200,10369500,6187200,17966400,2662700,43163700,7315200,4241500,32814500,8209800,8785500,6262600,70099000,94516000,1863300,5844100,8102300,14782200,10987600


### Merge with close price

In [11]:
res_merging = pd.DataFrame()
for symbol in symbols_stocks:
    for_merging = dataset_full.copy()
    for_merging = for_merging[['Date_get_data', 'Company']]
    for_merging = for_merging.drop_duplicates()
    tmp = pd.DataFrame(data_stocks_close[['Date', symbol]])
    tmp.columns = ['Date_get_data', 'Stock_price']
    tmp['Company'] = symbol
    for_merging = for_merging.merge(tmp, on=['Date_get_data', 'Company'], how='inner')
    res_merging = res_merging.append(for_merging)

In [12]:
dataset_full = dataset_full.merge(res_merging, on=['Date_get_data', 'Company'], how='left')

In [13]:
dataset_full.isnull().sum()

Contract Name             0
Strike                    0
Last Price                0
Bid                       0
Ask                       0
Change                    0
% Change                  0
Volume                    0
Open Interest             0
Implied Volatility        0
Company                   0
Date_get_data             0
Date_expiration           0
Time_to_maturity          0
r                         0
Stock_price           87010
dtype: int64

### Why nan?

Note that there is a column named Date_get_data. Sometimes I was getting data during weekends (meaning that I collected data on the previous working day compared to the date of getting data). Hence, the data was not available exactly on the data of collecting data, and we can just use ffill to fillna

In [14]:
set(dataset_full.loc[dataset_full['Stock_price'].isnull()]['Date_get_data'].values)

{numpy.datetime64('2024-02-19T00:00:00.000000000'),
 numpy.datetime64('2024-03-02T00:00:00.000000000'),
 numpy.datetime64('2024-03-29T00:00:00.000000000')}

In [15]:
# Fillna smartly
dataset_full['Stock_price'] = dataset_full.groupby('Company')['Stock_price'].apply(lambda x: x.ffill())

In [16]:
dataset_full.isnull().sum()

Contract Name         0
Strike                0
Last Price            0
Bid                   0
Ask                   0
Change                0
% Change              0
Volume                0
Open Interest         0
Implied Volatility    0
Company               0
Date_get_data         0
Date_expiration       0
Time_to_maturity      0
r                     0
Stock_price           0
dtype: int64

### Merge with volume

In [17]:
res_merging = pd.DataFrame()
for symbol in symbols_stocks:
    for_merging = dataset_full.copy()
    for_merging = for_merging[['Date_get_data', 'Company']]
    for_merging = for_merging.drop_duplicates()
    tmp = pd.DataFrame(data_stocks_volume[['Date', symbol]])
    tmp.columns = ['Date_get_data', 'Stock_volume']
    tmp['Company'] = symbol
    for_merging = for_merging.merge(tmp, on=['Date_get_data', 'Company'], how='inner')
    res_merging = res_merging.append(for_merging)

In [18]:
dataset_full = dataset_full.merge(res_merging, on=['Date_get_data', 'Company'], how='left')

In [19]:
dataset_full.isnull().sum()

Contract Name             0
Strike                    0
Last Price                0
Bid                       0
Ask                       0
Change                    0
% Change                  0
Volume                    0
Open Interest             0
Implied Volatility        0
Company                   0
Date_get_data             0
Date_expiration           0
Time_to_maturity          0
r                         0
Stock_price               0
Stock_volume          87010
dtype: int64

In [20]:
# Fillna smartly
dataset_full['Stock_volume'] = dataset_full.groupby('Company')['Stock_volume'].apply(lambda x: x.ffill())

In [21]:
dataset_full.isnull().sum()

Contract Name         0
Strike                0
Last Price            0
Bid                   0
Ask                   0
Change                0
% Change              0
Volume                0
Open Interest         0
Implied Volatility    0
Company               0
Date_get_data         0
Date_expiration       0
Time_to_maturity      0
r                     0
Stock_price           0
Stock_volume          0
dtype: int64

In [22]:
dataset_full.head()

,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Company,Date_get_data,Date_expiration,Time_to_maturity,r,Stock_price,Stock_volume
0,AAPL240223C00095000,95.0,86.97,87.1,87.6,0.00,-,1,7,177.34%,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
1,AAPL240223C00100000,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,173.44%,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
2,AAPL240223C00110000,110.0,83.50,72.1,72.65,0.00,-,-,1,148.05%,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
3,AAPL240223C00115000,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,135.94%,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
4,AAPL240223C00120000,120.0,64.40,62.1,62.65,0.00,-,6,3,124.61%,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0


In [23]:
# checked, units are OK
dataset_full['Implied Volatility'] = dataset_full['Implied Volatility'].str.replace('%', '')
dataset_full['Implied Volatility'] = dataset_full['Implied Volatility'].str.replace(',', '')
dataset_full['Implied Volatility'] = dataset_full['Implied Volatility'].astype(float)
dataset_full['Implied Volatility'] = dataset_full['Implied Volatility'] * 0.01
dataset_full

,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Company,Date_get_data,Date_expiration,Time_to_maturity,r,Stock_price,Stock_volume
0,AAPL240223C00095000,95.0,86.97,87.1,87.6,0.00,-,1,7,1.7734,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
1,AAPL240223C00100000,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,1.7344,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
2,AAPL240223C00110000,110.0,83.50,72.1,72.65,0.00,-,-,1,1.4805,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
3,AAPL240223C00115000,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,1.3594,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
4,AAPL240223C00120000,120.0,64.40,62.1,62.65,0.00,-,6,3,1.2461,AAPL,2024-02-16,2024-02-23,0.019178,0.04,182.309998,49701400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223321,INTC261218C00050000,50.0,5.70,5.55,5.8,-0.80,-12.31%,42,1420,0.4266,INTC,2024-04-12,2026-12-18,2.684932,0.04,35.689999,80139400.0
1223322,INTC261218C00055000,55.0,4.80,4.7,4.8,-0.60,-11.11%,73,31727,0.4229,INTC,2024-04-12,2026-12-18,2.684932,0.04,35.689999,80139400.0
1223323,INTC261218C00060000,60.0,3.95,3.8,4.05,-0.57,-12.61%,68,2618,0.4229,INTC,2024-04-12,2026-12-18,2.684932,0.04,35.689999,80139400.0
1223324,INTC261218C00065000,65.0,3.30,2.52,3.35,-0.47,-12.47%,22,304,0.4189,INTC,2024-04-12,2026-12-18,2.684932,0.04,35.689999,80139400.0


In [24]:
# It is sometimes possible that I got data the day after the option has expired
dataset_full.loc[dataset_full['Time_to_maturity'] < 0, 'Time_to_maturity'] = 0

In [25]:
dataset_full.to_csv('3_Clean_data_02_05.csv', index=None)

### Create our own volatility

It is not fair to use implied volatility, since it is estimated with the use of models (such as Black-Scholes). Hence, using implied volatility means using data leakage (and we will see that later). Since implied volatility will not be available on normal test data, we will create our own volatility by taking variance of stock dynamics of underlying assets

In [26]:
volatilites = pd.DataFrame(data_stocks_close.describe())
volatilites

,AAPL,ACN,AMZN,AVGO,BAC,CSCO,DG,DIS,GOOGL,INTC,JNJ,JPM,KO,LLY,LMT,MA,MCD,META,MRK,MSFT,NFLX,NVDA,ORCL,PEP,PFE,PG,PYPL,QCOM,SPY,TSLA,TSN,UNH,V,VZ,WMT
count,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000,1077.000000
mean,143.406722,285.874801,139.334368,581.873092,33.569721,48.969081,198.336175,125.938384,110.303173,45.699452,160.817809,136.770919,56.615952,314.446082,408.449842,355.621811,244.984577,262.946815,90.132012,272.318069,430.219229,246.307683,81.517837,159.493046,40.071164,141.621244,139.623166,128.697298,402.707911,207.996750,68.209285,433.094346,218.457558,48.436249,47.575258
std,33.712330,52.912602,27.876533,255.153258,6.772596,5.512201,38.062939,33.762623,25.999042,11.365241,11.279073,24.966170,5.536238,165.895893,45.636257,45.012721,33.242680,86.825670,16.466180,63.968597,123.444185,178.755734,21.107116,18.068930,7.866570,12.502686,78.007652,26.070639,56.942328,83.207548,13.303517,89.086229,24.827000,9.163205,4.848465
min,56.092499,143.690002,81.820000,167.869995,18.080000,33.200001,101.830002,79.320000,52.706501,24.900000,111.139999,79.029999,37.560001,119.050003,276.799988,203.300003,137.100006,88.910004,63.358780,135.419998,166.369995,49.099998,39.799999,103.930000,25.860001,97.699997,50.389999,60.910000,222.949997,24.081333,44.180000,194.860001,135.740005,30.670000,34.683334
25%,125.120003,253.649994,115.660004,437.850006,28.480000,45.160000,168.759995,96.610001,88.739998,35.459999,152.389999,118.300003,53.189999,183.000000,371.700012,330.230011,218.020004,198.449997,76.507637,225.750000,341.760010,132.767502,62.779999,143.289993,34.629982,135.240005,73.500000,114.019997,371.130005,166.350006,58.119999,347.809998,202.470001,39.020000,44.509998
50%,147.919998,287.019989,145.240005,517.440002,33.459999,48.910000,208.479996,116.309998,113.602501,47.560001,161.970001,138.729996,57.770000,269.000000,408.720001,354.440002,248.070007,265.859985,83.244278,265.019989,438.619995,186.720001,79.589996,164.300003,38.624287,142.850006,105.980003,129.479996,411.489990,219.600006,65.000000,469.649994,215.770004,51.959999,47.483334
75%,171.130005,320.779999,163.106506,631.090027,37.959999,53.060001,225.250000,151.940002,134.282501,54.430000,169.070007,154.770004,60.750000,365.769989,448.149994,376.630005,269.559998,321.839996,105.879997,316.380005,519.119995,288.850006,95.150002,173.149994,46.720001,150.960007,200.820007,144.880005,442.760010,260.019989,79.529999,505.660004,231.660004,56.450001,50.336666
max,198.110001,415.420013,189.050003,1407.010010,49.380001,63.959999,260.440002,201.910004,159.410004,68.470001,186.009995,200.300003,66.209999,792.280029,501.410004,488.640015,300.529999,527.340027,131.949997,429.369995,691.690002,950.020020,129.240005,196.119995,61.250000,164.210007,308.529999,189.279999,523.169983,409.970001,99.089996,555.150024,290.369995,61.740002,61.450001


In [27]:
# Note that we do not square standard deviation
# This is done according to the Black-Scholes paper
volatilites = volatilites.loc[volatilites.index == 'std']
volatilites = volatilites.T
volatilites = volatilites.reset_index()
volatilites.columns = ['Company', 'Volatility']
volatilites

,Company,Volatility
0,AAPL,33.712330
1,ACN,52.912602
2,AMZN,27.876533
3,AVGO,255.153258
4,BAC,6.772596
5,CSCO,5.512201
6,DG,38.062939
7,DIS,33.762623
8,GOOGL,25.999042
9,INTC,11.365241


In [28]:
# however, later on we will not use it 
# (we will calculate volatilities based on contract themselves, not the underlying stocks)
volatilites.to_csv('volatilities_companies.csv', index=None)

In [29]:
1 + 1

2

### Complete